In [5]:
# Fucntions 

In [6]:
# prompt: create fucntion to fecth the mdel test a body and return true or false
def predict_text(text, model, vectorizer):
    # Transform the input text using the fitted vectorizer
    text_vec = vectorizer.transform([text])
    # Predict the label
    prediction = model.predict(text_vec)
    # Calculate the probability of each class 
    probabilities = model.predict_proba(text_vec)
    # Return True if predicted label is spam (assuming spam is labeled as 1), otherwise False
    is_spam = prediction[0] == 1
    return is_spam, probabilities

def get_suspect_words(text, model, vectorizer, top_n=5):
    # Transform the input text using the fitted vectorizer
    text_vec = vectorizer.transform([text])

    # Get feature names (words) from the vectorizer
    feature_names = np.array(vectorizer.get_feature_names_out())

    # Get feature importances from the RandomForestClassifier
    feature_importances = model.feature_importances_

    # Get the nonzero feature indices in the transformed input
    word_indices = text_vec.nonzero()[1]

    # Get word importance scores
    word_importance = feature_importances[word_indices]

    # Pair words with their importance scores and sort them
    word_contributions = sorted(zip(feature_names[word_indices], word_importance), key=lambda x: abs(x[1]), reverse=True)

    # Return the top N suspect words
    return word_contributions[:top_n]

#load the model and vectorizer
import pickle
import numpy as np
# Load the trained model and vectorizer
model_path = '../app/model/model.pkl'
vectorizer_path = '../app/model/vectorizer.pkl'
with open(model_path, 'rb') as model_file:
    model = pickle.load(model_file)
with open(vectorizer_path, 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

In [7]:
# Example usage: Predict using title and body in the format: {subject}. Body: {body}
subject = "Your Email Title"
body_text = "Your email content goes here"
sample_text = f"{subject}. Body: {body_text}"
is_spam, probabilities = predict_text(sample_text, model, vectorizer)
print("Is Spam:", is_spam)
print("Probabilities:", probabilities)
top_words = get_suspect_words(sample_text, model, vectorizer, top_n=5)
print("Top Suspect Words:", top_words)
# Example usage Subject: {subject}. Body: {body}” so that the model sees the full context. 
sample_text = "Hay send me your other email so I can forward it to you" 
is_spam, probabilities = predict_text(sample_text, model, vectorizer)
print("Is Spam:", is_spam)
print("Probabilities:", probabilities)
top_words = get_suspect_words(sample_text, model, vectorizer, top_n=5)
print("Top Suspect Words:", top_words)

Is Spam: True
Probabilities: [[0.22 0.78]]
Top Suspect Words: [('body', np.float64(0.025705340059782873)), ('your', np.float64(0.005374586856843774)), ('here', np.float64(0.0024704972781055)), ('email', np.float64(0.0012545246879213418)), ('goes', np.float64(0.00010771780246092959))]
Is Spam: True
Probabilities: [[0.44 0.56]]
Top Suspect Words: [('your', np.float64(0.005374586856843774)), ('you', np.float64(0.004793338120540907)), ('to', np.float64(0.0034933589215651916)), ('it', np.float64(0.00339516750731521)), ('so', np.float64(0.0025861037610440496))]


# Using Fucntion

In [8]:
subject = "Your Email Title"
test_body = "We detected that your password has been exposed in a data breach on another platform while doing a routine check for your security. You should reset your Duolingo password immediately, using a unique password not used elsewhere."
test_text = f"{subject}. Body: {test_body}"
is_spam, percentages = predict_text(test_text, model, vectorizer)
suspect_words = get_suspect_words(test_text, model, vectorizer)

if is_spam:
    confidence = max(percentages[0])
    print("The email is a phishing email. With a probability of", round(confidence *100, 2), "%")
else:
    confidence = max(percentages[0])
    print("The email is not a phishing email. With a probability of",  round(confidence *100, 2), "%")

print("Top 5 suspect words:", suspect_words)

The email is a phishing email. With a probability of 51.0 %
Top 5 suspect words: [('body', np.float64(0.025705340059782873)), ('not', np.float64(0.006446988484846362)), ('that', np.float64(0.006394237694236446)), ('on', np.float64(0.005749832373025211)), ('your', np.float64(0.005374586856843774))]
